In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten
from tensorflow.keras.models import Sequential, Model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib


file_dir = r'C:/Users/sarah/Documents/Python Scripts/7243 Final Project/'   

labels = pd.read_csv(file_dir + 'hemorrhage-labels.csv')
display(labels)

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000012eaf,0,0,0,0,0,0
1,ID_000039fa0,0,0,0,0,0,0
2,ID_00005679d,0,0,0,0,0,0
3,ID_00008ce3c,0,0,0,0,0,0
4,ID_0000950d7,0,0,0,0,0,0
...,...,...,...,...,...,...,...
752798,ID_ffff82e46,0,0,0,0,0,0
752799,ID_ffff922b9,1,0,0,1,0,0
752800,ID_ffffb670a,1,0,0,0,1,0
752801,ID_ffffcbff8,0,0,0,0,0,0


In [2]:
"""
In the conditions list, the in-line comment explains which categorical dummy variable that relates to the column.
These are explained here as well: 
normal case = 0 (i.e. no hemorrhage)
epidural = 1; intraparenchymal = 2; intraventricular = 3; subarachnoid = 4; subdural = 5; multiple = 6
"""
conditions = [
    # the 'multiple' case (i.e. has a 1 in two or more columns, not including 'any' column)
    # .gt(1) :=  test for greater than 1
    labels[['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']].sum(1).gt(1),   # 6
    labels['epidural'] == 1,   # 1
    labels['intraparenchymal'] == 1,   # 2
    labels['intraventricular'] == 1,   # 3
    labels['subarachnoid'] == 1,   #4
    labels['subdural'] == 1,   #5
]
outputs = [6, 1, 2, 3, 4, 5]

# np.nan characteristic sets the default case (i.e. the 'normal' or no hemorrhage case as NaN so we can remove easily)
# this could be changed to a zero if necessary 
dummy = np.select(conditions, outputs, 0)
dummy = pd.Series(dummy)
dummy.name = "category"

labels_cat = pd.concat([labels, dummy], axis=1)
labels_cat['Image'] =  labels_cat['Image'] + '.jpg'

display(labels_cat)

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,category
0,ID_000012eaf.jpg,0,0,0,0,0,0,0
1,ID_000039fa0.jpg,0,0,0,0,0,0,0
2,ID_00005679d.jpg,0,0,0,0,0,0,0
3,ID_00008ce3c.jpg,0,0,0,0,0,0,0
4,ID_0000950d7.jpg,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
752798,ID_ffff82e46.jpg,0,0,0,0,0,0,0
752799,ID_ffff922b9.jpg,1,0,0,1,0,0,3
752800,ID_ffffb670a.jpg,1,0,0,0,1,0,4
752801,ID_ffffcbff8.jpg,0,0,0,0,0,0,0


In [3]:
import glob, os

test = []
imagedir = []
imagename = []
c = 0

for filename in glob.iglob(file_dir + 'renders/**', recursive=True):
    if os.path.isfile(filename):
        test.append(filename)
        dirname, basename = os.path.split(test[c])
        imagedir.append(dirname)
        imagename.append(basename)
        c += 1 # filter dirs

In [4]:
img = pd.DataFrame([imagedir, imagename], index=['image_path', 'Image']).T.explode('Image')
display(img)

,image_path,Image
0,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_000edbf38.jpg
1,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_00178eb80.jpg
2,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_001bb2c00.jpg
3,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_0026de01c.jpg
4,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_004966e2d.jpg
...,...,...
373279,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_fff24b028.jpg
373280,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_fff49fa03.jpg
373281,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_fff4c8969.jpg
373282,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_fff4dc2f1.jpg


In [5]:
#img_nodups = img.drop_duplicates()
clabels_dups = pd.merge(img, labels_cat, on=["Image"], how = "right")
clabels_na = clabels_dups.drop_duplicates(subset = "Image")
clabels_na.reset_index(drop = True, inplace = True)

display(clabels_na)

,image_path,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,category
0,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_000012eaf.jpg,0,0,0,0,0,0,0
1,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_000039fa0.jpg,0,0,0,0,0,0,0
2,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_00005679d.jpg,0,0,0,0,0,0,0
3,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_00008ce3c.jpg,0,0,0,0,0,0,0
4,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_0000950d7.jpg,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
752798,NaN,ID_ffff82e46.jpg,0,0,0,0,0,0,0
752799,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_ffff922b9.jpg,1,0,0,1,0,0,3
752800,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_ffffb670a.jpg,1,0,0,0,1,0,4
752801,NaN,ID_ffffcbff8.jpg,0,0,0,0,0,0,0


In [6]:
clabels = clabels_na.dropna()
clabels.reset_index(drop = True, inplace = True)
display(clabels)

,image_path,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,category
0,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_000012eaf.jpg,0,0,0,0,0,0,0
1,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_000039fa0.jpg,0,0,0,0,0,0,0
2,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_00005679d.jpg,0,0,0,0,0,0,0
3,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_00008ce3c.jpg,0,0,0,0,0,0,0
4,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_0000950d7.jpg,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
116076,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_fffc60817.jpg,1,0,1,1,0,0,6
116077,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_fffd00949.jpg,1,0,0,0,1,0,4
116078,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_fffe2edb8.jpg,1,0,1,1,0,0,6
116079,C:/Users/sarah/Documents/Python Scripts/7243 F...,ID_ffff922b9.jpg,1,0,0,1,0,0,3


In [ ]:
#skipping for now for memory purposes
size = []

# save the image path in its own dataframe then enumerate through the path list NOT the labels.
# then index and reindex the labels

m = 0
n = 0
counter = 0

for path, image in zip(clabels.image_path, clabels.Image):
    #print(n)
    if len(np.mean(matplotlib.image.imread(path + '/' + image),axis=2).reshape(-1)) == 262144:
        m += 1
    else:
        size.append(m)
        display(path, image)
        print(m)
        counter += 1
        m += 1
        continue
    n += 1
print(counter)    

In [ ]:
#skipping for now for memory purposes
final = clabels.drop(size)
final.reset_index(drop = True, inplace = True)
display(final)

In [7]:
import random
## Need seeding to select same sample each time
random.seed(42)

img_list = list(clabels.Image)
cat_list= list(clabels.category)

normal=[]
epid=[]
intrap =[]
intrav=[]
suba =[]
subd =[]
multi =[]

for i in range(len(cat_list)):
    if cat_list[i]==0:
        normal.append(img_list[i])
    elif cat_list[i]==1:
        epid.append(img_list[i])
    elif cat_list[i]==2:
        intrap.append(img_list[i])
    elif cat_list[i]==3:
        intrav.append(img_list[i])
    elif cat_list[i]==4:
        suba.append(img_list[i])
    elif cat_list[i]==5:
        subd.append(img_list[i])
    elif cat_list[i]==6:
        multi.append(img_list[i])
    
    
normal_sample=random.sample(normal, k = 500)
epid_sample=random.sample(epid, k = 500)
intrap_sample=random.sample(intrap, k = 500)
intrav_sample=random.sample(intrav, k = 500)
suba_sample=random.sample(suba, k = 500)
subd_sample=random.sample(subd, k = 500)
multi_sample=random.sample(multi, k = 500)

img_sample= normal_sample + epid_sample + intrap_sample + intrav_sample + suba_sample + subd_sample+ multi_sample

y=[]

print(len(img_sample))
print(img_sample[3])

3500
ID_026f9b78b.jpg


In [8]:
import glob, os
from PIL import Image
from numpy import asarray

imgs = []
img_names=[]
c=0

valid_images = [".jpg",".gif",".png",".tga"]


In [10]:
for i in range(len(test)):
    try:
        dirname, basename = os.path.split(test[i])
        if basename in img_names:
            continue
    except:
        print("not a file")
        continue
    if basename in img_sample:
        try:
            imgs.append(Image.open(os.path.join(test[i])))
            print(len(imgs))
            img_names.append(basename)
        except:
            print(basename, "cannot be added")
    else:
        continue
            

print(len(imgs))
print(len(img_names))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064


In [ ]:
print(len(imgs))
print(len(img_names))

In [11]:
y=[]

for basename in img_names:
    if basename in normal_sample:
        y.append(0)
    elif basename in epid_sample:
        y.append(1)
    elif basename in intrap_sample:
        y.append(2)
    elif basename in intrav_sample:
        y.append(3)
    elif basename in suba_sample:
        y.append(4)
    elif basename in subd_sample:
        y.append(5)
    elif basename in multi_sample:
        y.append(6)

print(len(y))

3500


In [12]:
for i in range (0,7):
    print(i, y.count(i))

0 500
1 500
2 500
3 500
4 500
5 500
6 500


In [13]:
print(imgs[0].size)

(512, 512)


In [14]:
#This converts each image from a jpg into an array

from numpy import asarray
import skimage
from skimage import measure

print(len(imgs))

print(type(imgs[0]))

downsample = 4

for i in range(len(imgs)):
    imgs[i]=np.asarray(imgs[i])
    imgs[i] = imgs[i]/255
    r = skimage.measure.block_reduce(imgs[i][:, :, 0],
                                 (downsample, downsample),
                                 np.mean)
    g = skimage.measure.block_reduce(imgs[i][:, :, 1],
                                 (downsample, downsample),
                                 np.mean)
    b = skimage.measure.block_reduce(imgs[i][:, :, 2],
                                 (downsample, downsample),
                                 np.mean)
    imgs[i] = np.stack((r, g, b), axis=-1)

print(type(imgs[0]))
print(np.shape(imgs[0]))  #This is the right shape for a CNN

3500
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'numpy.ndarray'>
(128, 128, 3)


In [17]:
print(type(imgs[0]))
print(np.shape(imgs[0]))  #This is the right shape for a CNN
imgs_final=[]

for i in range(len(imgs)):
    if np.shape(imgs[i])==(128,128,3):
        imgs_final.append(imgs[i])
    else:
        print(i)
        
print(len(imgs_final))

<class 'numpy.ndarray'>
(128, 128, 3)
3129
3499


In [18]:
del(y[3129])

print(len(y))

from sklearn.model_selection import train_test_split
imgs_train, imgs_test, y_train, y_test = train_test_split(imgs_final, 
                                                    np.asarray(y), test_size=0.2)

imgs_train= np.array(imgs_train)
imgs_test= np.array(imgs_test)

print(imgs_train.shape)
print(imgs_test.shape)

print(y_train.shape)
print(y_test.shape)

3499
(2799, 128, 128, 3)
(700, 128, 128, 3)
(2799,)
(700,)


In [19]:
from tensorflow.keras import utils

y_train = utils.to_categorical(y_train,7)
y_test = utils.to_categorical(y_test,7)

imgs_train = imgs_train.astype('float32')
imgs_test = imgs_test.astype('float32')
imgs_train /= 255
imgs_test /= 255

print(imgs_train.shape)
print(imgs_test.shape)
print(y_train.shape)
print(y_test.shape)

(2799, 128, 128, 3)
(700, 128, 128, 3)
(2799, 7)
(700, 7)


In [20]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, callbacks

early_stopping_cb = keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
)

In [102]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.layers import Conv2D, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (5,5),
                 padding='valid',
                 input_shape=(128, 128, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(units=490, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=7, activation='softmax'))

opt = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9, decay=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        3

In [22]:
epochs = 200

history = model.fit(imgs_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=32,
                    verbose=1,
                    validation_data=(imgs_test, y_test), callbacks=early_stopping_cb)

Epoch 1/200
88/88 [==============================] - 63s 714ms/step - loss: 1.7717 - accuracy: 0.2715 - val_loss: 1538.7570 - val_accuracy: 0.1657
Epoch 2/200
88/88 [==============================] - 63s 715ms/step - loss: 1.4947 - accuracy: 0.3101 - val_loss: 2192.6353 - val_accuracy: 0.1657
Epoch 3/200
88/88 [==============================] - 63s 714ms/step - loss: 1.4451 - accuracy: 0.3233 - val_loss: 2084.3635 - val_accuracy: 0.1657
Epoch 4/200
88/88 [==============================] - 63s 714ms/step - loss: 1.4140 - accuracy: 0.3373 - val_loss: 2041.4495 - val_accuracy: 0.1657
Epoch 5/200
88/88 [==============================] - 68s 771ms/step - loss: 1.3695 - accuracy: 0.3558 - val_loss: 1705.7781 - val_accuracy: 0.1657
Epoch 6/200
88/88 [==============================] - 64s 724ms/step - loss: 1.3570 - accuracy: 0.3551 - val_loss: 1350.6198 - val_accuracy: 0.1657
Epoch 7/200
88/88 [==============================] - 65s 742ms/step - loss: 1.3397 - accuracy: 0.3687 - val_loss: 1170

88/88 [==============================] - 53s 604ms/step - loss: 1.2209 - accuracy: 0.4416 - val_loss: 1.4737 - val_accuracy: 0.3971
Epoch 58/200
88/88 [==============================] - 53s 607ms/step - loss: 1.2258 - accuracy: 0.4319 - val_loss: 1.3652 - val_accuracy: 0.4571
Epoch 59/200
88/88 [==============================] - 56s 637ms/step - loss: 1.2128 - accuracy: 0.4577 - val_loss: 1.3540 - val_accuracy: 0.4443
Epoch 60/200
88/88 [==============================] - 53s 602ms/step - loss: 1.2152 - accuracy: 0.4437 - val_loss: 1.4232 - val_accuracy: 0.4000
Epoch 61/200
88/88 [==============================] - 55s 624ms/step - loss: 1.2128 - accuracy: 0.4534 - val_loss: 1.3757 - val_accuracy: 0.4571
Epoch 62/200
88/88 [==============================] - 52s 589ms/step - loss: 1.2178 - accuracy: 0.4334 - val_loss: 1.3621 - val_accuracy: 0.4529
Epoch 63/200
88/88 [==============================] - 52s 588ms/step - loss: 1.2191 - accuracy: 0.4452 - val_loss: 1.3728 - val_accuracy: 0.427

Let's see if we get any better results by taking out the multi-data

In [83]:
# Trying again with a larger dataset

import random
## Need seeding to select same sample each time
random.seed(42)

img_list = list(clabels.Image)
cat_list= list(clabels.category)

normal=[]
epid=[]
intrap =[]
intrav=[]
suba =[]
subd =[]
multi =[]

for i in range(len(cat_list)):
    if cat_list[i]==0:
        normal.append(img_list[i])
    elif cat_list[i]==1:
        epid.append(img_list[i])
    elif cat_list[i]==2:
        intrap.append(img_list[i])
    elif cat_list[i]==3:
        intrav.append(img_list[i])
    elif cat_list[i]==4:
        suba.append(img_list[i])
    elif cat_list[i]==5:
        subd.append(img_list[i])
    elif cat_list[i]==6:
        multi.append(img_list[i])
    
    
normal_sample=random.sample(normal, k = 1000)
epid_sample=random.sample(epid, k = 1000)
intrap_sample=random.sample(intrap, k = 1200)
intrav_sample=random.sample(intrav, k = 1200)
suba_sample=random.sample(suba, k = 1200)
subd_sample=random.sample(subd, k = 1200)
multi_sample=random.sample(multi, k = 1200)

img_sample= normal_sample + epid_sample + intrap_sample + intrav_sample + suba_sample + subd_sample + multi_sample

y=[]

print(len(img_sample))
print(img_sample[3])

8000
ID_026f9b78b.jpg


In [84]:
import glob, os
from PIL import Image
from numpy import asarray

imgs = []
img_names=[]
c=0

valid_images = [".jpg",".gif",".png",".tga"]


In [85]:
for i in range(len(test)):
    try:
        dirname, basename = os.path.split(test[i])
        if basename in img_names:
            continue
    except:
        continue
    if basename in img_sample:
        try:
            imgs.append(Image.open(os.path.join(test[i])))
            img_names.append(basename)
        except:
            print(basename + "cannnot be added")
    else:
        continue
            

print(len(imgs))
print(len(img_names))

8000
8000


In [86]:
y=[]

for basename in img_names:
    if basename in normal_sample:
        y.append(0)
    elif basename in epid_sample:
        y.append(1)
    elif basename in intrap_sample:
        y.append(2)
    elif basename in intrav_sample:
        y.append(3)
    elif basename in suba_sample:
        y.append(4)
    elif basename in subd_sample:
        y.append(5)
    elif basename in multi_sample:
        y.append(6)

print(len(y))

8000


In [87]:
from numpy import asarray
import skimage
from skimage import measure

print(len(imgs))

print(type(imgs[0]))

downsample = 4

for i in range(len(imgs)):
    imgs[i]=np.asarray(imgs[i])
    imgs[i] = imgs[i]/255
    r = skimage.measure.block_reduce(imgs[i][:, :, 0],
                                 (downsample, downsample),
                                 np.mean)
    g = skimage.measure.block_reduce(imgs[i][:, :, 1],
                                 (downsample, downsample),
                                 np.mean)
    b = skimage.measure.block_reduce(imgs[i][:, :, 2],
                                 (downsample, downsample),
                                 np.mean)
    imgs[i] = np.stack((r, g, b), axis=-1)

print(type(imgs[0]))
print(np.shape(imgs[0]))  #This is the right shape for a CNN

8000
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'numpy.ndarray'>
(128, 128, 3)


In [88]:
print(type(imgs[0]))
print(np.shape(imgs[0]))  #This is the right shape for a CNN
imgs_final=[]

for i in range(len(imgs)):
    if np.shape(imgs[i])==(128,128,3):
        imgs_final.append(imgs[i])
    else:
        del(y[i])
        
print(len(imgs_final))

<class 'numpy.ndarray'>
(128, 128, 3)
7997


In [89]:
from sklearn.model_selection import train_test_split
imgs_train, imgs_test, y_train, y_test = train_test_split(imgs_final, 
                                                    np.asarray(y), test_size=0.2)

imgs_train= np.array(imgs_train)
imgs_test= np.array(imgs_test)

print(imgs_train.shape)
print(imgs_test.shape)

print(y_train.shape)
print(y_test.shape)

(6397, 128, 128, 3)
(1600, 128, 128, 3)
(6397,)
(1600,)


In [90]:
from tensorflow.keras import utils

y_train = utils.to_categorical(y_train,7)
y_test = utils.to_categorical(y_test,7)

imgs_train = imgs_train.astype('float32')
imgs_test = imgs_test.astype('float32')
imgs_train /= 255
imgs_test /= 255

print(imgs_train.shape)
print(imgs_test.shape)
print(y_train.shape)
print(y_test.shape)

(6397, 128, 128, 3)
(1600, 128, 128, 3)
(6397, 7)
(1600, 7)


In [108]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [64]:
model = Sequential()

model.add(Conv2D(32, (5,5),
                 padding='valid',
                 input_shape=(128, 128, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(64, (3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
#model.add(Dense(units=490, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=7, activation='softmax'))

opt = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9, decay=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        3

In [66]:
epochs = 100

history = model.fit(imgs_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=32,
                    verbose=1,
                    validation_data=(imgs_test, y_test))

Epoch 1/100
175/175 [==============================] - 105s 600ms/step - loss: 1.4079 - accuracy: 0.3569 - val_loss: 356.0670 - val_accuracy: 0.1364
Epoch 2/100
175/175 [==============================] - 108s 615ms/step - loss: 1.3075 - accuracy: 0.3876 - val_loss: 338.9831 - val_accuracy: 0.1364
Epoch 3/100
175/175 [==============================] - 107s 609ms/step - loss: 1.2654 - accuracy: 0.4153 - val_loss: 237.2032 - val_accuracy: 0.1364
Epoch 4/100
175/175 [==============================] - 115s 659ms/step - loss: 1.2511 - accuracy: 0.4305 - val_loss: 104.8721 - val_accuracy: 0.1364
Epoch 5/100
175/175 [==============================] - 109s 624ms/step - loss: 1.2381 - accuracy: 0.4425 - val_loss: 8.8883 - val_accuracy: 0.1657
Epoch 6/100
175/175 [==============================] - 112s 637ms/step - loss: 1.2178 - accuracy: 0.4503 - val_loss: 11.3031 - val_accuracy: 0.1679
Epoch 7/100
175/175 [==============================] - 108s 615ms/step - loss: 1.2126 - accuracy: 0.4623 - va

175/175 [==============================] - 107s 614ms/step - loss: 1.1301 - accuracy: 0.4982 - val_loss: 1.2759 - val_accuracy: 0.4414
Epoch 57/100
175/175 [==============================] - 107s 614ms/step - loss: 1.1310 - accuracy: 0.5020 - val_loss: 1.2697 - val_accuracy: 0.4457
Epoch 58/100
175/175 [==============================] - 110s 626ms/step - loss: 1.1290 - accuracy: 0.5023 - val_loss: 1.2558 - val_accuracy: 0.4443
Epoch 59/100
175/175 [==============================] - 107s 613ms/step - loss: 1.1327 - accuracy: 0.4980 - val_loss: 1.2592 - val_accuracy: 0.4464
Epoch 60/100
175/175 [==============================] - 107s 611ms/step - loss: 1.1307 - accuracy: 0.5082 - val_loss: 1.2385 - val_accuracy: 0.4579
Epoch 61/100
175/175 [==============================] - 113s 648ms/step - loss: 1.1313 - accuracy: 0.5077 - val_loss: 1.2428 - val_accuracy: 0.4586
Epoch 62/100
175/175 [==============================] - 109s 621ms/step - loss: 1.1254 - accuracy: 0.5027 - val_loss: 1.2164 

In [129]:
keras.backend.clear_session()
tf.random.set_seed(10)
np.random.seed(10)

model = Sequential()

model.add(Conv2D(32, (5,5),
                 padding='valid',
                 input_shape=(128, 128, 3),
                 activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(64, (3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
#model.add(Dense(units=490, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(units=7, activation='softmax'))

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        3

In [ ]:
epochs = 100

history = model.fit(imgs_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=32,
                    verbose=1,
                    validation_data=(imgs_test, y_test))

#I made it so the learning rate does not decay

Epoch 1/100
 72/200 [=========>....................] - ETA: 1:14 - loss: 1.7506 - accuracy: 0.3147

In [104]:
# Next steps: Save the weights if good, repeat process to upload different set of 8000 images, run again
# Also need to attempt segmentation side - use pretrained UNet

#This website seems like it will be very helpful: https://segmentation-models.readthedocs.io/en/latest/tutorial.html
# My guess is that our y- values will have to change to the coordinates that indicate where the hemorrhage is located